In [1]:
# load segmentation data

import numpy as np
import h5py

# set segmentation filename
seg_file = "/home/srujanm/cerebellum/data/gt48nm_cropped_relabeled_pf_0014.h5"

seg = np.array(h5py.File(seg_file,'r')['main'])
seg_name = "helloworld" # give the segmentation a name of your choice
res = (30, 48, 48) # resolution of segmentation data in (nm, nm, nm) in order (Z,Y,X)
print seg.shape

/home/srujanm/anaconda2/envs/em/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(90, 540, 488)


In [2]:
# creates a .meta file with segmentation properties, used globally by skeletonization routines
import os

if not os.path.isdir('./meta/'):
    os.mkdir('./meta/')
meta = open("./meta/" + seg_name +'.meta', "w")
meta.write("# resolution in nm\n")
meta.write("%dx%dx%d\n"%(res[2], res[1], res[0]))
meta.write("# grid size\n")
meta.write("%dx%dx%d\n"%(seg.shape[2], seg.shape[1], seg.shape[0]))
meta.close()

In [3]:
def check_ids(seg):
    """
    checks if all IDs in segmentation are consecutive
    Args:
        seg (ndarray): segmentation data
    """
    seg_ids = np.unique(seg)
    max_id = np.max(seg_ids)
    n_ids = len(seg_ids)
    try:
        assert max_id == n_ids-1
    except:
        missing_ids = np.sort(np.array(list(set(range(max_id+1)).difference(set(seg_ids)))))
        print "Error! Labels in segmentation are not consecutive. %d IDs are missing"%(len(missing_ids))
        print missing_ids

In [4]:
def relabel(seg):
    """
    Relabels a segmentation such that max ID = # objects
    Args:
        seg (ndarray): 3D segmentation
    Returns:
        seg_relabeled (ndarray)
    """
    seg_ids = np.unique(seg)
    n_ids = len(seg_ids)
    max_id = np.max(seg_ids)
    if max_id == n_ids-1:
        return seg
    missing_ids = np.sort(np.array(list(set(range(max_id+1)).difference(set(seg_ids)))))
    seg_relabel = seg
    for i in range(len(missing_ids)):
        if i==len(missing_ids)-1:
            ids_to_correct = range(missing_ids[i]+1, max_id+1)
        else:
            ids_to_correct = range(missing_ids[i]+1, missing_ids[i+1])
        for j in ids_to_correct:
            seg_relabel[seg==j] = j-(i+1) #TODO (Jeff): speed this up using object-wise bounding boxes
    return seg_relabel

In [5]:
# run skeletonization routines
import os 

from ibex.transforms.seg2seg import DownsampleMapping
from ibex.skeletonization.generate_skeletons import TopologicalThinning, FindEndpointVectors, FindEdges

dsmpl_res = (80,80,80) # downsample segmentation to this resolution in (nm, nm, nm) before skeletonization
if not os.path.isdir('./skeletons/'): # skeleton results are saved to ./skeletons/<seg_name>/
    os.mkdir('./skeletons/')
seg = seg.astype(np.int64)
check_ids(seg) 
# if check_ids fails, relabel your segmentation using the relabel function
# seg = relabel(seg)

# results will be saved to ./skeletons/<seg_name>
DownsampleMapping(seg_name, seg, output_resolution=dsmpl_res)
TopologicalThinning(seg_name, seg, skeleton_resolution=dsmpl_res)
FindEndpointVectors(seg_name, skeleton_resolution=dsmpl_res)
FindEdges(seg_name, skeleton_resolution=dsmpl_res)

Downsampling to resolution (80, 80, 80) in 0.459539175034 seconds
Topological thinning time for (80, 80, 80): 7.22298812866
Endpoint vector time for (80, 80, 80): 1.63487100601
Edge finding time for (80, 80, 80): 0.310307025909


In [6]:
# read skeletons
from ibex.utilities.dataIO import ReadSkeletons
skeletons = ReadSkeletons(seg_name, downsample_resolution=dsmpl_res, read_edges=True)
print len(skeletons)

1003


In [7]:
# generate 3D plot of skeleton for object of interest
# result will be saved to ./skeletons/<seg_name>
skel_id = 347
skeletons[skel_id].save_image('./skeletons/'+seg_name+'/')